In [1]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim, os, pickle
import tokenize
import numpy as np
import pandas as pd

In [2]:
##### take some data
df_train = pd.read_csv('../../../../Data/Raw/train_clean.csv',encoding='utf-8')
qid_stack = df_train['qid1'].append(df_train['qid2'])
df_train_naExc = df_train[~df_train.isnull().any(axis=1)]
df_buf1 = df_train_naExc[['qid1','question1']]
df_buf2 = df_train_naExc[['qid2','question2']]
df_buf1.columns = ['qid','question']
df_buf2.columns = ['qid','question']
df_stack = df_buf1.append(df_buf2)
df_stack_dupExc = df_stack.drop_duplicates()
test_buf = df_stack_dupExc['question']#.head(1000)#.iloc[5000:7000]#head(1000)

In [3]:
pkl_file = open('LDA_pickle_New_20170520_50topic_ldamodel_test_traing.pkl', 'rb')
ldamodel_dump = pickle.load(pkl_file)
pkl_file.close()

pkl_file1 = open('Py2_dictionary.pkl', 'rb')
dictionary = pickle.load(pkl_file1)
pkl_file.close()
#print(ldamodel_dump.keys())
##### for predict 
#ldamodel = ldamodel_dump['lda_model']
#dictionary = ldamodel_dump['dictionary']
##### for build the lda-model by yourself 
#corpus = ldamodel_dump['encoding_corpus']

In [12]:
############# some tools
######### Tokenize the corpus
tokenizer = RegexpTokenizer(r'\w+')
# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
def tokenize_doc(new_corpus):
    new_corpus
    texts = []
    ###
    ### Progressing bar setting
    totall_size = len(new_corpus)
    counter = 0
    progress = 0
    # loop through document list
    for i in new_corpus:    
#         ### Monitor the progress
#         if progress%(1000) ==0:
#             bar = "[" + int(progress/1000)*"|"+">"+ (10-int(progress/1000))*"-" + str((progress/100)) + "%" "]"
#             print(bar)
#         #### count
#         counter += 1
#         progress = int(10000*(counter/totall_size))

        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)

        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        
        # stem tokens
#         stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        stemmed_tokens = []
        for i in stopped_tokens:
            try:
                stemmed_tokens = stemmed_tokens + [p_stemmer.stem(i)]
            except:
                print(i)
        # add tokens to list            
        texts.append(stemmed_tokens)
    return texts

######  predict the topic
def topic_predict(new_corpus, model, dictionary):
    tokenize_buf = tokenize_doc([new_corpus])
    doc_bow = [dictionary.doc2bow(text) for text in tokenize_buf]
    que_vec = [item for itemList in doc_bow for item in itemList]
    topic_vec = model[que_vec]

    word_count_array = np.empty((len(topic_vec), 2), dtype = np.object)
    for i in range(len(topic_vec)):
        word_count_array[i, 0] = topic_vec[i][0]
        word_count_array[i, 1] = topic_vec[i][1]

    idx = np.argsort(word_count_array[:, 1])
    idx = idx[::-1]
    word_count_array = word_count_array[idx]

    final = []
    final = model.print_topic(word_count_array[0, 0], len(word_count_array))#model.print_topic(20, 20)# model.print_topic(word_count_array[0, 0], len(word_count_array)) # model.print_topic(word_count_array[0, 0], len(word_count_array))#1)

    question_topic = final.split('*') ## as format is like "probability * topic"
    return question_topic

In [13]:
###### check the topic stack
print(ldamodel_dump.print_topics(num_topics=10, num_words=4))

[(19, u'0.277*"india" + 0.074*"countri" + 0.022*"s" + 0.022*"educ"'), (25, u'0.114*"job" + 0.101*"univers" + 0.070*"look" + 0.044*"major"'), (38, u'0.067*"servic" + 0.049*"can" + 0.045*"exist" + 0.040*"idea"'), (11, u'0.158*"quora" + 0.105*"question" + 0.058*"answer" + 0.031*"anim"'), (3, u'0.305*"use" + 0.068*"word" + 0.039*"can" + 0.035*"sentenc"'), (39, u'0.066*"day" + 0.057*"1" + 0.049*"3" + 0.049*"5"'), (34, u'0.054*"man" + 0.039*"choos" + 0.037*"purpos" + 0.035*"girlfriend"'), (20, u'0.127*"t" + 0.065*"want" + 0.053*"feel" + 0.051*"love"'), (31, u'0.072*"sex" + 0.063*"data" + 0.038*"law" + 0.035*"big"'), (6, u'0.097*"live" + 0.092*"us" + 0.036*"usa" + 0.033*"parent"')]


In [14]:
###### Predict a topic
pivot = 1
print(test_buf.iloc[pivot])
print(topic_predict(test_buf.iloc[pivot], ldamodel_dump, dictionary))

What is the story of Kohinoor (Koh-i-Noor) Diamond 
[u'0.058', u'"car" + 0.034', u'"interview"']


In [15]:
###### Predict a topic
print("="*90)
for i in range(100,1000,100):
    pivot = i
    print("Question:\n"+test_buf.iloc[pivot]+"\n")
    print(topic_predict(test_buf.iloc[pivot], ldamodel_dump, dictionary))
    print("="*90)

Question:
Will there really be any war between India and Pakistan over the Uri attack  What will be its effects 

[u'0.047', u'"trump" + 0.047', u'"2016" + 0.045', u'"come" + 0.043', u'"product" + 0.039', u'"will" + 0.037', u'"win"']
Question:
What is the easiest way to become a billionaire($) 

[u'0.221', u'"make" + 0.139', u'"much" + 0.128', u'"money" + 0.061', u'"can"']
Question:
How can we acquire a positive morality 

[u'0.052', u'"call" + 0.044', u'"follow" + 0.040', u'"instagram" + 0.037', u'"can"']
Question:
Why do dreams look so real 

[u'0.200', u'"mean" + 0.094', u'"say" + 0.046', u'"die"']
Question:
What makes a programmer "good "

[u'0.052', u'"call" + 0.044', u'"follow"']
Question:
What should I do for Web design 

[u'0.070', u'"account"']
Question:
What is the difference between a turkey and a chicken 

[u'0.053', u'"cultur" + 0.040', u'"differ"']
Question:
What is the benefit of going Walking every morning 

[u'0.170', u'"time" + 0.111', u'"book" + 0.092', u'"possibl" +